In [2]:
import time
import requests
import csv
from selenium import webdriver
from bs4 import BeautifulSoup
import os
import sys
from selenium.webdriver.common.keys import Keys
import chromedriver_autoinstaller
import warnings
import pandas as pd
import numpy as np

In [4]:
df = pd.read_csv('게임_리스트_수집_950.csv')
del df["Unnamed: 0"]
df = df.dropna()
df.isna().sum()#제거됨
df.columns

Index(['game_name', 'game_id'], dtype='object')

In [5]:
id_list = df.game_id.values
id_list

array(['812140', '1029690', '1172620', '1238810', '1296610', '960170',
       '1172380', '958260', '892970', '518790', '920210', '1593500',
       '453480', '977950', '1435790', '570', '394360', '779340', '250900',
       '1966900', '105600', '255710', '306130', '323190', '550', '281990',
       '1761390', '1517290', '1203220', '1384160', '242760', '761890',
       '1665460', '613100', '1721470', '1594940', '489830', '1151340',
       '251570', '1147560', '1290000', '1277400', '1621690', '427520',
       '962130', '270880', '686810', '1477590', '1635450', '1794680',
       '1250410', '1938090', '582160', '1253920', '594650', '1466860',
       '1407200', '594570', '1016920', '1091500', '1062090', '444200',
       '880940', '1142710', '292030', '1451940', '872410', '740130',
       '477160', '1190000', '674020', '1151640', '493520', '632470',
       '1630280', '275850', '367520', '767560', '646570', '1739010',
       '1145360', '1217060', '377160', '1498570', '8500', '1084600',
       '1

In [6]:
# 크롤링
import requests
import time
def get_reviews(appid, params={'json':1}):
        url = 'https://store.steampowered.com/appreviews/'
        response = requests.get(url=url+str(appid), params=params, headers={'User-Agent': 'Mozilla/5.0'})
        return response.json()

#https://partner.steamgames.com/doc/store/getreviews 참고    
def get_n_reviews(appid, n=100):#appid는 해당게임의 상점번호
    reviews = []
    cursor = '*'
    params = {
        'json' : 1,
        'filter' : 'all',#필터 없이
        'language' : 'koreana',#한국어로된 리뷰
        'day_range' : 9223372036854775807,#전체기간의 리뷰
        'review_type' : 'all',
        # 'purchase_type' : 'all'
    }

    while n > 0:
        time.sleep(1)#2초씩 쉬면서 크롤링
        params['cursor'] = cursor.encode()
        params['num_per_page'] = min(100, n)
        n -= 100

        response = get_reviews(appid, params)
        cursor = response['cursor']
        reviews += response['reviews']

        if len(response['reviews']) < 100: break

    return reviews

In [ ]:
reviews = []
for i in id_list:
    reviews.append(get_n_reviews(i, 20))# (크롤링할 게임번호, 리뷰 갯수)

In [9]:

df2 = pd.DataFrame()

for i in range(len(reviews)):
  data = pd.DataFrame.from_dict(reviews[i])
  data['gameid'] = id_list[i]
  df2 = df2.append(data)

df2.reset_index(drop = True)

,recommendationid,author,language,review,timestamp_created,timestamp_updated,voted_up,votes_up,votes_funny,weighted_vote_score,comment_count,steam_purchase,received_for_free,written_during_early_access,gameid,timestamp_dev_responded,developer_response
0,49255781,"{'steamid': '76561198332096156', 'num_games_ow...",koreana,《Assassin's Creed® Odyssey》\n\n『게임 내적 부분』\n\n「...,1.551235e+09,1.551235e+09,True,217.0,32.0,0.919744014739990234,19.0,True,False,False,812140,NaN,NaN
1,59106609,"{'steamid': '76561198256644273', 'num_games_ow...",koreana,처음 캐릭터 고를때 여자로 하세요 남캐로 하면 수영할때 부랄 덜렁거리는거 계속 봐야함,1.575359e+09,1.575359e+09,True,117.0,160.0,0.906629264354705811,17.0,True,False,False,812140,NaN,NaN
2,54227713,"{'steamid': '76561198078511630', 'num_games_ow...",koreana,300시간 좀안되게 플레이한결과 굉장히 만족스러운 게임이었음.\n\n후반부로 갈수록...,1.563802e+09,1.563802e+09,False,158.0,17.0,0.89348602294921875,6.0,True,False,False,812140,NaN,NaN
3,45120052,"{'steamid': '76561198100058825', 'num_games_ow...",koreana,스포일러 없는 후기 및 팁 \n\n어쌔신크리드 시리즈 최초로 여캐 남캐 선택이 추가...,1.538755e+09,1.643881e+09,True,103.0,10.0,0.887289464473724365,0.0,True,False,False,812140,NaN,NaN
4,47211346,"{'steamid': '76561198002545632', 'num_games_ow...",koreana,딸기맛 우유가 그냥 딸기향 나는 첨가물이 들어간 달짝지근한 우유인것처럼\n어쌔신 크...,1.543592e+09,1.543602e+09,True,150.0,84.0,0.884254813194274902,9.0,True,False,False,812140,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15490,34743257,"{'steamid': '76561198366382348', 'num_games_ow...",koreana,nice game one love!,1.504557e+09,1.504557e+09,True,6.0,2.0,0.583333313465118408,0.0,True,False,False,10,NaN,NaN
15491,45492750,"{'steamid': '76561198223161840', 'num_games_ow...",koreana,전설의 시작,1.540776e+09,1.540776e+09,True,5.0,1.0,0.581749081611633301,0.0,True,False,False,10,NaN,NaN
15492,96297707,"{'steamid': '76561199111338301', 'num_games_ow...",koreana,카온보단 100배 나은듯,1.627140e+09,1.627140e+09,True,2.0,0.0,0.545454561710357666,0.0,True,False,False,10,NaN,NaN
15493,42392405,"{'steamid': '76561198090445278', 'num_games_ow...",koreana,추억팔이 할려했더만 서버에 사람이 없네..,1.527716e+09,1.527716e+09,True,2.0,1.0,0.545454561710357666,0.0,True,False,False,10,NaN,NaN


In [15]:
df2 = df2[['gameid','review']]
df2

,gameid,review
0,812140,게임 내적 부분 게임성 마약겜 굉장히 재미있었음 게임으로서의 가치는 있음 음 미묘함...
1,812140,처음 캐릭터 고를때 여자로 하세요 남캐로 하면 수영할때 부랄 덜렁거리는거 계속 봐야함
2,812140,시간 좀안되게 플레이한결과 굉장히 만족스러운 게임이었음 후반부로 갈수록 조금 단조로...
3,812140,스포일러 없는 후기 및 팁 어쌔신크리드 시리즈 최초로 여캐 남캐 선택이 추가되었습니...
4,812140,딸기맛 우유가 그냥 딸기향 나는 첨가물이 들어간 달짝지근한 우유인것처럼 어쌔신 크리...
...,...,...
13,10,솔직히 안사도 될 작품이지만 샀다 왜 방 카운터에서 카스 주세요 하던 시절이 생각나...
14,10,아 다시 날아올라라 이여 그럴일은 없겠지만 그 시절로 돌아가고싶다 세일할때 게임 수...
16,10,전설의 시작
17,10,카온보단 배 나은듯


In [16]:
#전처리 ( 공백제거 따옴표 제거 특문 제거 ,최대문자설정, 좌우공백 제거)
df2['review'] = df2['review'] \
  .replace(r'[^가-힣 ]', ' ', regex=True) \
  .replace("'", '') \
  .replace(r'\s+', ' ', regex=True) \
  .str.strip() \
  .str[:255]

df2.head()

,gameid,review
0,812140,게임 내적 부분 게임성 마약겜 굉장히 재미있었음 게임으로서의 가치는 있음 음 미묘함...
1,812140,처음 캐릭터 고를때 여자로 하세요 남캐로 하면 수영할때 부랄 덜렁거리는거 계속 봐야함
2,812140,시간 좀안되게 플레이한결과 굉장히 만족스러운 게임이었음 후반부로 갈수록 조금 단조로...
3,812140,스포일러 없는 후기 및 팁 어쌔신크리드 시리즈 최초로 여캐 남캐 선택이 추가되었습니...
4,812140,딸기맛 우유가 그냥 딸기향 나는 첨가물이 들어간 달짝지근한 우유인것처럼 어쌔신 크리...


In [18]:
# 공백 리뷰 제거
df2 = df2[df2['review'].str.strip().astype(bool)]

len(df2) # 제거되고난뒤 리뷰 갯수

14968

In [19]:
df2.rename(columns={'gameid':'game_id'},inplace=True)
df2

,game_id,review
0,812140,게임 내적 부분 게임성 마약겜 굉장히 재미있었음 게임으로서의 가치는 있음 음 미묘함...
1,812140,처음 캐릭터 고를때 여자로 하세요 남캐로 하면 수영할때 부랄 덜렁거리는거 계속 봐야함
2,812140,시간 좀안되게 플레이한결과 굉장히 만족스러운 게임이었음 후반부로 갈수록 조금 단조로...
3,812140,스포일러 없는 후기 및 팁 어쌔신크리드 시리즈 최초로 여캐 남캐 선택이 추가되었습니...
4,812140,딸기맛 우유가 그냥 딸기향 나는 첨가물이 들어간 달짝지근한 우유인것처럼 어쌔신 크리...
...,...,...
13,10,솔직히 안사도 될 작품이지만 샀다 왜 방 카운터에서 카스 주세요 하던 시절이 생각나...
14,10,아 다시 날아올라라 이여 그럴일은 없겠지만 그 시절로 돌아가고싶다 세일할때 게임 수...
16,10,전설의 시작
17,10,카온보단 배 나은듯


In [20]:
데이터셋 = pd.merge(df,df2 ,how='outer',on = 'game_id')
데이터셋= 데이터셋[['game_name','game_id','review']]

In [21]:
데이터셋.to_csv("데이터셋1.csv", index=False)